# Introduction

After joining the ingredients_df with their respective food_id in the density_db, we need to do a similar matching of their respective food_portion entries.

# Process

This will follow a very similar process as with the food_df joining.

There are a few points to keep in mind whilst doing this. We aren't actually trying to match the units, instead it's the type of ingredient eg. cinnamon ground vs. whole stick. The units themselves matter up to their 'unit_type' ie. weight, volume, or whole. If the ingredient is given in weight, we don't need this density information. Whereas if it's given in volume, we need to match it with a volume portion in order to calculate its density. The same goes for a 'whole' unit, to get an idea of its portion size.

# Setup

In [1]:
#| default_exp density.portion_match

In [2]:
#| export
from pyprojroot import here
root = here()
import sys
sys.path.append(str(root))

In [3]:
#| export
import pandas as pd
import numpy as np
import seaborn as sns

import nltk
import spacy
from spacy.matcher import Matcher
from spacy.util import filter_spans

from thefuzz import fuzz

from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import torch
from sklearn.metrics.pairwise import cosine_similarity

import json
from itertools import groupby
import re
import string

from tqdm import tqdm
tqdm.pandas()

from recipe_dataset.utils.utils import *

In [4]:
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
ingredients_df = pd.read_feather('../../data/local/recipe/partial/ingredients/0.feather')
food_df = pd.read_feather('../../data/local/density/full/food/0.feather')
food_portion_df = pd.read_feather('../../data/local/density/full/food_portion/0.feather')
food_ids = pd.read_feather('../../data/local/density/partial/food_ids/0.feather')

In [6]:
ingredients_df[['comment', 'unit']] = ingredients_df[['comment', 'unit']].fillna('')
ingredients_df = ingredients_df.join(food_ids)

In [7]:
food_portion_df.rename({'remainder_tags':'unit_remainders'}, inplace=True, axis=1)

In [8]:
#| export
with open(f'{root}/config/unit_conversions.json') as f:
    unit_list = json.load(f)

# Food Portion Selection

The main function selecting the appropriate food_portion.

Following the two principles highlighted above, we are first looking at the *unit_type* for a match. The process for weight/volume types are self-explanatory, whereas portions are where more complexity arrises.

In [9]:
ingredient = ingredients_df.iloc[0]
ingredient

name.name                                           butter
name.description                          land lake butter
quantity                                             0.667
unit                                                  cups
comment                                                   
preparation                                       softened
ingredient_string    2/3 cup Land O Lakes Butter, softened
unit_tags                                            [cup]
unit_remainders                                         []
unit_type                                           volume
food_id                                          2345703.0
Name: (1746116, 0), dtype: object

#todo - if no unit AND quantity then that is simply a serving/portion of the item.

In [10]:
#| export
def select_food_portion(ingredient, food_portion_df, debug=False):

    if pd.isnull(ingredient['food_id']): return pd.NA

    portions = food_portion_df.loc[ingredient['food_id'],:]
    debug_info = {
        'tag_filter_results': [],
        'description_search_words': '',
        'description_search_results': [],
    }

    if ingredient['unit_type'] == 'volume':
        # if volume given, find (any) other volume portion
        searched = portions.index[portions['unit_tags'].apply(lambda x: any([i in unit_list['volume'].keys() for i in x]))]
        debug_info['tag_filter_results'].append({'filter': 'volume', 'indices': searched})
        if not searched.empty: portions = portions.loc[searched]
    elif ingredient['unit_type'] == 'weight':
        # if weight given, we don't need any matches
        searched = portions.index[portions['unit_tags'].apply(lambda x: any([i in unit_list['weight'].keys() for i in x]))]
        debug_info['tag_filter_results'].append({'filter': 'weight', 'indices': searched})
        if not searched.empty: portions = portions.loc[searched]
    else:
        # if volume nor weight given
        # test: removing results with units
        searched = portions.index[portions['unit_tags'].apply(lambda x: not any([i not in unit_list['portion'].keys() for i in x]))]
        if not searched.empty: portions = portions.loc[searched]
        # first we want to check if units match
        if len(ingredient['unit_tags']):
            searched = portions.index[portions['unit_tags'].apply(lambda x: any([i in ingredient['unit_tags'] for i in x]))]
            debug_info['tag_filter_results'].append({'filter': 'portion_unit_match', 'indices': searched})
            if not searched.empty: portions = portions.loc[searched]
        # then check if remainders match
        if len(ingredient['unit_remainders']):
            searched = portions.index[portions['unit_remainders'].apply(lambda x: any([i in ingredient['unit_remainders'] for i in x]))]
            debug_info['tag_filter_results'].append({'filter': 'portion_unit_remainder_match', 'indices': searched})
            if not searched.empty: portions = portions.loc[searched]
        # if 'whole' portion specified
        if not len(ingredient['unit_tags']) or any([i == 'whole' for i in ingredient['unit_tags']]):
            # if 'whole' explicitly specified:
            searched = portions.index[portions['unit_tags'].apply(lambda x: any([i == 'whole' for i in x]))]
            debug_info['tag_filter_results'].append({'filter': 'portion_whole_explicit', 'indices': searched})
            if not searched.empty: portions = portions.loc[searched]
            else:
                searched = portions.index[(portions['unit_tags'].str.len() == 0)]
                if not searched.empty: 
                    portions = portions.loc[searched]
                    # select the one with the largest gram weight (#todo factor in `amount` too)
                    searched = portions.index[portions['gram_weight'] == portions['gram_weight'].max()]
                    if not searched.empty: portions = portions.loc[searched]

    # searching for ingredient terms
    search_cols = ['name.description', 'comment', 'unit']
    search_words = []
    for col, value in ingredient[search_cols].items():
        if not pd.isnull(value): 
            search_words.extend(ingredient[col].split(' '))
        else:
            if col == 'unit': 
                search_words.extend(['whole', 'serving', 'portion']) 

    search_words.reverse()

    debug_info['description_search_words'] = search_words

    for search_word in search_words:
        for col in ['unit_tags', 'unit_remainders']:
            searched = portions.index[portions[col].apply(lambda x: search_word in x)]
            if not searched.empty:
                portions = portions.loc[searched]
                debug_info['description_search_results'].append({'search_word': search_word, 'indices': searched})

    # finding globals
    default_words = ['raw', 'regular', 'normal', 'medium']
    for word in default_words:
        searched = portions['description'].str.contains(word)
        if searched.any():
            portions = portions[searched]
            debug_info['tag_filter_results'].append({'filter': 'default_units', 'indices': searched.index})
    
    return portions.iloc[0].name if not debug else (portions.iloc[0].name, debug_info)

In [11]:
ingredient

name.name                                           butter
name.description                          land lake butter
quantity                                             0.667
unit                                                  cups
comment                                                   
preparation                                       softened
ingredient_string    2/3 cup Land O Lakes Butter, softened
unit_tags                                            [cup]
unit_remainders                                         []
unit_type                                           volume
food_id                                          2345703.0
Name: (1746116, 0), dtype: object

In [12]:
food_portion_df.loc[ingredient['food_id']]

,seq_num,amount,gram_weight,description,unit_tags,unit_remainders,unit_type,portion_amount,portion_unit
id,,,,,,,,,
287267,1,1.0,224.0,cup,[cup],[],volume,NaN,None
287268,2,1.0,14.0,tablespoon,[tablespoon],[],volume,NaN,None
287269,3,1.0,7.0,pat,[],[pat],portion,NaN,None
287270,4,1.0,5.0,individual container,[],"[individual, container]",portion,NaN,None
287271,5,1.0,7.0,slice bread/roll,[piece],"[bread, roll]",portion,NaN,None
287272,6,1.0,14.0,regular sandwich,[whole],[sandwich],portion,NaN,None
287273,7,1.0,28.0,large sandwich,[whole],[sandwich],portion,NaN,None


In [13]:
selected_portion, debug_info = select_food_portion(ingredient, food_portion_df, True)
food_portion_df.loc[ingredient['food_id'], selected_portion]

seq_num                 1
amount                1.0
gram_weight         224.0
description           cup
unit_tags           [cup]
unit_remainders        []
unit_type          volume
portion_amount        NaN
portion_unit         None
Name: (2345703, 287267), dtype: object

In [14]:
debug_info

{'tag_filter_results': [{'filter': 'volume',
   'indices': Index([287267, 287268], dtype='int64', name='id')}],
 'description_search_words': ['cups', '', 'butter', 'lake', 'land'],
 'description_search_results': []}

In [15]:
ingredients_df[ingredients_df['food_id'].isna()]

name.name                   name.description  \
recipe  ingredient                                                             
1828339 3           long grain brown rice              long grain brown rice   
        6                   dried apricot                      dried apricot   
        7                         sultana                            sultana   
1703    3                       mincemeat                          mincemeat   
1262123 0                        linguine                           linguine   
...                                   ...                                ...   
599284  1                          potato  frozen shredded hash brown potato   
1357213 3                        cinnamin                           cinnamin   
2006319 2                       raspberry                          raspberry   
        3                      blackberry                         blackberry   
        4                       blueberry                          blueberry   

                    quantity    unit comment        preparation  \
recipe  ingredient                                                
1828339 3              175.0       g                       <NA>   
        6              100.0       g             finely chopped   
        7               50.0       g                       <NA>   
1703    3                1.0     jar                       <NA>   
1262123 0                4.0      oz                       <NA>   
...                      ...     ...     ...                ...   
599284  1                1.0      lb          completely thawed   
1357213 3                0.5    tsps                       <NA>   
2006319 2                8.0  ounces                       <NA>   
        3                8.0  ounces                       <NA>   
        4                8.0  ounces                       <NA>   

                                                    ingredient_string  \
recipe  ingredient                                                      
1828339 3                                 175 g long grain brown rice   
        6                        100 g dried apricots, finely chopped   
        7                                               50 g sultanas   
1703    3                                             1 jar mincemeat   
1262123 0                                               4 oz linguine   
...                                                               ...   
599284  1           1 lb. frozen shredded hash brown potatoes, com...   
1357213 3                                           1/2 tsp. cinnamin   
2006319 2                                        8 ounces raspberries   
        3                                       8 ounces blackberries   
        4                                        8 ounces blueberries   

                     unit_tags unit_remainders unit_type  food_id  
recipe  ingredient                                                 
1828339 3               [gram]              []    weight      NaN  
        6               [gram]              []    weight      NaN  
        7               [gram]              []    weight      NaN  
1703    3                   []           [jar]   portion      NaN  
1262123 0              [ounce]              []    weight      NaN  
...                        ...             ...       ...      ...  
599284  1              [pound]              []    weight      NaN  
1357213 3           [teaspoon]              []    volume      NaN  
2006319 2              [ounce]              []    weight      NaN  
        3              [ounce]              []    weight      NaN  
        4              [ounce]              []    weight      NaN  

[200 rows x 11 columns]

In [16]:
food_portion_ids = ingredients_df.progress_apply(select_food_portion, args=(food_portion_df,), axis=1)

  7%|▋         | 177/2450 [00:00<00:02, 885.15it/s]

100%|██████████| 2450/2450 [00:02<00:00, 870.36it/s]


In [17]:
food_portion_ids = food_portion_ids.rename('food_portion_id')

In [18]:
ingredients_df = ingredients_df.join(food_portion_ids)

## Evaluating

Visualising the selections, along with their possible options.

In [19]:
pd.set_option('display.max_rows', None)

In [20]:
ingredients_df.join(food_df['description'], on='food_id').join(food_portion_df[['description','gram_weight']], lsuffix='(food)', rsuffix='(portion)', on=['food_id', 'food_portion_id']).merge(food_portion_df.reset_index().set_index('fdc_id')[['gram_weight', 'description']], left_on='food_id', suffixes=('(portion_selected)', '(portion_options)'), right_index=True).head(100)

name.name                name.description  \
recipe  ingredient                                                              
1746116 0                              butter                land lake butter   
        0                              butter                land lake butter   
        0                              butter                land lake butter   
        0                              butter                land lake butter   
        0                              butter                land lake butter   
        0                              butter                land lake butter   
        0                              butter                land lake butter   
        1                               sugar                           sugar   
        1                               sugar                           sugar   
        1                               sugar                           sugar   
        1                               sugar                           sugar   
        1                               sugar                           sugar   
        2                                 egg                   land lake egg   
        2                                 egg                   land lake egg   
        2                                 egg                   land lake egg   
        2                                 egg                   land lake egg   
        2                                 egg                   land lake egg   
        2                                 egg                   land lake egg   
        3                             vanilla                         vanilla   
        3                             vanilla                         vanilla   
        4                               flour               all-purpose flour   
        5                             caramel                         caramel   
        5                             caramel                         caramel   
        5                             caramel                         caramel   
        5                             caramel                         caramel   
        5                             caramel                         caramel   
        5                             caramel                         caramel   
        5                             caramel                         caramel   
        6                               cream  land lake heavy whipping cream   
        6                               cream  land lake heavy whipping cream   
        6                               cream  land lake heavy whipping cream   
        6                               cream  land lake heavy whipping cream   
        7                               pecan                      pecan half   
        7                               pecan                      pecan half   
        7                               pecan                      pecan half   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet chocolate chip   
        8           semi-sweet chocolate chip  real semi-sweet choc

This all looks fine, something more difficult is those with no unit tags:

In [21]:
sample_df = ingredients_df[(ingredients_df['unit_tags'].str.len() == 0)].sample(20, random_state=1330)
sample_df.join(food_df['description'], on='food_id').join(food_portion_df[['description','gram_weight']], lsuffix='(food)', rsuffix='(portion)', on=['food_id', 'food_portion_id']).merge(food_portion_df.reset_index().set_index('fdc_id')[['description','gram_weight']], left_on='food_id', suffixes=('(portion_selected)', '(portion_options)'), right_index=True).head(40)

name.name  \
recipe  ingredient                            
1627293 5                               egg   
        5                               egg   
        5                               egg   
        5                               egg   
        5                               egg   
        5                               egg   
855127  0                          eggplant   
        0                          eggplant   
972254  8             italian seasoning mix   
        8             italian seasoning mix   
101786  0           instant vanilla pudding   
        0           instant vanilla pudding   
1369744 4                             onion   
        4                             onion   
        4                             onion   
        4                             onion   
303741  1                      swiss cheese   
        1                      swiss cheese   
2058996 6                             lower   
        6                             lower   
        6                             lower   
2076879 1                              salt   
        1                              salt   
        1                              salt   
        1                              salt   
1408069 4                              sage   
        4                              sage   
55647   1                          macaroni   
221805  0                        green bean   
        0                        green bean   
1723278 7                             thyme   
        7                             thyme   
640253  0                cranberry cocktail   
        0                cranberry cocktail   
        0                cranberry cocktail   
1748614 3                               nut   
        3                               nut   
1342486 9                            garlic   
        9                            garlic   
        9                            garlic   

                                       name.description  quantity    unit  \
recipe  ingredient                                                          
1627293 5                                 egg separated       2.0           
        5                                 egg separated       2.0           
        5                                 egg separated       2.0           
        5                                 egg separated       2.0           
        5                                 egg separated       2.0           
        5                                 egg separated       2.0           
855127  0                                      eggplant       1.0           
        0                                      eggplant       1.0           
972254  8           dried italian seasoning mix mixture      <NA>           
        8           dried italian seasoning mix mixture      <NA>           
101786  0                   pkg instant vanilla pudding       2.0           
        0                   pkg instant vanilla pudding       2.0           
1369744 4                                         onion       1.0           
        4                                         onion       1.0           
        4                                         onion       1.0           
        4                                         onion       1.0           
303741  1                                  swiss cheese      <NA>           
        1                                  swiss cheese      <NA>           
2058996 6             lower-sodium stoned wheat cracker      <NA>           
        6             lower-sodium stoned wheat cracker      <NA>           
        6             lower-sodium stoned wheat cracker      <NA>           
2076879 1                                   salt pepper       1.0           
        1                                   salt pepper       1.0           
        1                                   salt pepper       1.0           
        1                                   salt pepper  

Here we want to find cases where ingredients with volume measurements don't have a volume portion match. This is significant because we won't be able to get a weight measurement.

In [22]:
from ast import literal_eval

In [23]:
results_df = ingredients_df.join(food_portion_df, on=['food_id', 'food_portion_id'], rsuffix='(portion)')

results_df[results_df['unit_tags'].apply(lambda x: any([t in unit_list['volume'].keys() for t in x])) & ~(results_df['unit_tags(portion)'].apply(lambda x: any([t in unit_list['volume'].keys() for t in x]) if isinstance(x, np.ndarray) else True))]

,,name.name,name.description,quantity,unit,comment,preparation,ingredient_string,unit_tags,unit_remainders,unit_type,...,food_portion_id,seq_num,amount,gram_weight,description,unit_tags(portion),unit_remainders(portion),unit_type(portion),portion_amount,portion_unit
recipe,ingredient,,,,,,,,,,,,,,,,,,,,,
1781719,3,light molasses,light molasses,1.0,teaspoon,,<NA>,1 teaspoon light molasses,[teaspoon],[],volume,...,278155,3,1.0,30.0,medium,[whole],[],portion,NaN,None
165596,5,dark molasses,dark molasses,0.667,cups,,<NA>,2/3 cup dark molasses,[cup],[],volume,...,278155,3,1.0,30.0,medium,[whole],[],portion,NaN,None


In [24]:
food_df[food_df['description'].str.lower().str.contains('molasses')]

,data_type,description,description_list,description_length,description_list_length,default_word_count,exclusion_word_count,volume_exists,portion_exists
fdc_id,,,,,,,,,
172724,sr_legacy_food,"Cookies, molasses","[cooky, molasses]",17,2,0,0,False,True
2343099,survey_fndds_food,"Bread, cornmeal and molasses","[bread, cornmeal molasses]",28,2,0,0,False,True
2343100,survey_fndds_food,"Bread, cornmeal and molasses, toasted","[bread, cornmeal molasses, toasted]",37,3,0,0,False,True
2343424,survey_fndds_food,"Cookie, molasses","[cookie, molasses]",16,2,0,0,False,True


We can forgive these entries. They are occassional blips of foods that are too specific eg. bread, cookies. #TODO: This might be able to be filtered out in the food_df, using the 'density_exists' field as guidance.

# Saving

In [25]:
food_portion_ids.to_frame().to_feather('../../data/local/density/partial/food_portion_ids/0.feather')

In [26]:
from nbdev import nbdev_export; nbdev_export()